# Equalização de Histograma Interativa para Imagens de Microplásticos

## O que é CLAHE?

**CLAHE (Contrast Limited Adaptive Histogram Equalization)** é uma versão avançada da equalização de histograma:

### Equalização de Histograma Padrão:
- Aplica equalização globalmente em toda a imagem
- Pode amplificar excessivamente o ruído em regiões uniformes
- Pode perder detalhes locais

### Melhorias do CLAHE:
- **Adaptativo**: Divide a imagem em pequenos blocos e aplica equalização localmente
- **Contraste Limitado**: Corta picos do histograma para prevenir super-amplificação
- **Melhor Controle de Ruído**: Reduz amplificação de ruído em áreas uniformes
- **Preserva Detalhes Locais**: Mantém estruturas finas em imagens de microplásticos

### Parâmetros:
- **Limite de Corte (Clip Limit)**: Valores maiores = mais contraste (mas mais ruído)
- **Tamanho do Bloco (Tile Size)**: Blocos menores = mais adaptação local (mas mais computação)

### Funções OpenCV Utilizadas:
- **`cv2.imread()`**: Carrega imagem do disco
- **`cv2.cvtColor()`**: Converte entre espaços de cor (BGR→RGB, RGB→GRAY)
- **`cv2.equalizeHist()`**: Equalização de histograma padrão
- **`cv2.createCLAHE()`**: Cria objeto CLAHE com parâmetros específicos
- **`clahe.apply()`**: Aplica CLAHE na imagem em escala de cinza

Este notebook demonstra equalização de histograma interativa com comparação colorida.

In [1]:
# Importação das bibliotecas necessárias
import cv2  # OpenCV para processamento de imagens
import numpy as np  # NumPy para operações com arrays
import matplotlib.pyplot as plt  # Matplotlib para visualização
from PIL import Image  # PIL para manipulação adicional de imagens
import os  # OS para operações com sistema de arquivos
from ipywidgets import interact, FloatSlider, Dropdown, Checkbox, IntSlider  # Widgets interativos
from IPython.display import display  # Display para Jupyter

In [2]:
# Carregamento das imagens disponíveis no diretório de dados
data_dir = '../data'  # Diretório onde estão as imagens
supported_formats = ('.jpg', '.jpeg', '.png', '.tiff', '.tif')  # Formatos suportados

image_files = []  # Lista para armazenar nomes dos arquivos
if os.path.exists(data_dir):  # Verifica se o diretório existe
    # Filtra apenas arquivos com extensões suportadas
    image_files = [f for f in os.listdir(data_dir) 
                   if f.lower().endswith(supported_formats)]

if not image_files:
    print("Nenhuma imagem encontrada no diretório de dados")
else:
    print(f"Encontradas {len(image_files)} imagens: {image_files}")

Encontradas 26 imagens: ['P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'PCB1.1_LK_Q5.jpg', 'P1A_T1_LD_Q5_100X.jpg', 'CNN_P1_L1_T1_M1_LG_Q8_150x.jpg', 'PCB1.1_LK_Q7.jpg', 'NIVEL 2 (BR1_1CAMP_LO_Q6_125X).jpg', 'PCB1.1_LM_.jpg', 'NIVEL 5 (P1D_T1_LI_Q11_63X).jpg', 'NIVEL 4 (P1D_T2_LJ_Q10_150X).jpg', 'P1A_T1_LC_Q6_100X.jpg', 'CNN_P1_L1_T1_M1_LH_Q3_63x.jpg', 'CNN_P1_L1_T1_M1_LI_Q2_50x.jpg', 'P1A_T1_LG_Q8_150X (1).jpg', 'PCB1.1.jpg', 'NIVEL 1 (BR3_1CAMP_LC_Q6_100X).jpg', 'NIVEL 3 (P1D_T3_LH_Q8_100X (2)).jpg', 'PCB1.1LB_Q9.jpg', 'CNN_P1_L1_T1_M1_LG_Q5_80x.jpg', 'PCB1.1_LK_Q10.jpg', 'CNN_P1_L1_T1_M1_LH_Q9_F1_150x.jpg', 'PCB1.1_LB_Q6.jpg', 'CNN_P1_L1_T1_M1_LC_Q9_100x.jpg', 'CNN_P1_L1_T1_M1_LG_Q11_100x.jpg', 'CNN_P1_L1_T1_M1_LI_Q6_125x.jpg', 'P1A_T1_LI_Q11_40X.jpg']


In [3]:
def adaptive_histogram_equalization(image_path, clip_limit=2.0, tile_size=8, apply_clahe=True):
    """
    Aplica equalização de histograma com CLAHE opcional
    
    Parâmetros:
    - image_path: caminho para a imagem
    - clip_limit: limite de corte para CLAHE (controla amplificação)
    - tile_size: tamanho dos blocos para processamento local
    - apply_clahe: se True usa CLAHE, se False usa equalização padrão
    
    Retorna:
    - img_rgb: imagem original colorida
    - gray: imagem em escala de cinza
    - equalized: imagem equalizada
    """
    # Carrega a imagem (OpenCV carrega em formato BGR)
    img = cv2.imread(image_path)
    
    # Converte de BGR para RGB (formato padrão para visualização)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Converte para escala de cinza (necessário para equalização)
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    
    if apply_clahe:
        # Cria objeto CLAHE com parâmetros especificados
        # clipLimit: controla o quanto o contraste pode ser amplificado
        # tileGridSize: define o tamanho da grade de blocos (tile_size x tile_size)
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=(tile_size, tile_size))
        
        # Aplica CLAHE na imagem em escala de cinza
        equalized = clahe.apply(gray)
    else:
        # Aplica equalização de histograma padrão (global)
        equalized = cv2.equalizeHist(gray)
    
    return img_rgb, gray, equalized

In [4]:
def interactive_histogram_equalization(image_name, clip_limit, tile_size, apply_clahe):
    """
    Equalização de histograma interativa com comparação colorida, escala de cinza e equalizada
    
    Parâmetros:
    - image_name: nome do arquivo de imagem selecionado
    - clip_limit: limite de corte para CLAHE
    - tile_size: tamanho dos blocos para CLAHE
    - apply_clahe: usar CLAHE (True) ou equalização padrão (False)
    """
    if not image_files:
        print("Nenhuma imagem disponível")
        return
    
    # Constrói o caminho completo para a imagem
    image_path = os.path.join(data_dir, image_name)
    
    # Processa a imagem com os parâmetros especificados
    original_color, original_gray, equalized = adaptive_histogram_equalization(
        image_path, clip_limit, tile_size, apply_clahe)
    
    # Cria figura com 8 subplots organizados em 2 linhas e 4 colunas
    plt.figure(figsize=(18, 10))
    
    # LINHA SUPERIOR: Imagens
    
    # Subplot 1: Imagem original colorida
    plt.subplot(2, 4, 1)
    plt.imshow(original_color)  # Mostra imagem RGB original
    plt.title('Original Colorida')
    plt.axis('off')  # Remove eixos para melhor visualização
    
    # Subplot 2: Imagem original em escala de cinza
    plt.subplot(2, 4, 2)
    plt.imshow(original_gray, cmap='gray')  # cmap='gray' para escala de cinza
    plt.title('Original Escala de Cinza')
    plt.axis('off')
    
    # Subplot 3: Imagem equalizada
    plt.subplot(2, 4, 3)
    plt.imshow(equalized, cmap='gray')
    method = 'CLAHE' if apply_clahe else 'Padrão'  # Define método usado
    plt.title(f'Equalizada {method}')
    plt.axis('off')
    
    # Subplot 4: Comparação lado a lado (antes | depois)
    plt.subplot(2, 4, 4)
    # np.hstack concatena imagens horizontalmente
    comparison = np.hstack([original_gray, equalized])
    plt.imshow(comparison, cmap='gray')
    plt.title('Antes | Depois')
    plt.axis('off')
    
    # LINHA INFERIOR: Histogramas e parâmetros
    
    # Subplot 5: Histograma da imagem original
    plt.subplot(2, 4, 5)
    # .ravel() achata a matriz 2D em 1D para o histograma
    # 256 bins para valores de 0 a 255 (8 bits)
    plt.hist(original_gray.ravel(), 256, [0,256], alpha=0.7, color='blue')
    plt.title('Histograma Original')
    plt.xlabel('Intensidade')
    plt.ylabel('Frequência')
    
    # Subplot 6: Histograma da imagem equalizada
    plt.subplot(2, 4, 6)
    plt.hist(equalized.ravel(), 256, [0,256], alpha=0.7, color='red')
    plt.title('Histograma Equalizado')
    plt.xlabel('Intensidade')
    plt.ylabel('Frequência')
    
    # Subplot 7: Sobreposição dos histogramas para comparação
    plt.subplot(2, 4, 7)
    # alpha=0.5 torna as barras semi-transparentes para sobreposição
    plt.hist(original_gray.ravel(), 256, [0,256], alpha=0.5, label='Original', color='blue')
    plt.hist(equalized.ravel(), 256, [0,256], alpha=0.5, label='Equalizada', color='red')
    plt.title('Sobreposição de Histogramas')
    plt.xlabel('Intensidade')
    plt.ylabel('Frequência')
    plt.legend()  # Mostra legenda
    
    # Subplot 8: Painel de parâmetros
    plt.subplot(2, 4, 8)
    # Adiciona texto com os parâmetros atuais
    # transform=plt.gca().transAxes usa coordenadas relativas (0-1)
    plt.text(0.1, 0.8, f'Método: {method}', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.6, f'Limite de Corte: {clip_limit}', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.4, f'Tamanho do Bloco: {tile_size}x{tile_size}', fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.1, 0.2, f'Imagem: {image_name}', fontsize=10, transform=plt.gca().transAxes)
    plt.title('Parâmetros')
    plt.axis('off')
    
    # Ajusta layout para evitar sobreposição
    plt.tight_layout()
    plt.show()

In [ ]:
# Widget interativo para equalização de histograma
if image_files:
    interact(interactive_histogram_equalization,
             # Dropdown para seleção de imagem
             image_name=Dropdown(options=image_files, description='Imagem:'),
             # Slider para limite de corte (0.5 a 10.0)
             clip_limit=FloatSlider(min=0.5, max=10.0, step=0.5, value=2.0, 
                                  description='Limite de Corte:'),
             # Slider para tamanho do bloco (2 a 16, apenas pares)
             tile_size=IntSlider(min=2, max=16, step=2, value=8, 
                               description='Tamanho do Bloco:'),
             # Checkbox para ativar/desativar CLAHE
             apply_clahe=Checkbox(value=True, description='Usar CLAHE'))
else:
    print("Adicione imagens ao diretório ../data/ para usar a equalização interativa")

interactive(children=(Dropdown(description='Imagem:', options=('P1A_T1_LE_Q7_100X.jpg', 'PCB1.1.LO_Q8.jpg', 'P…